<a href="https://colab.research.google.com/github/melzismn/Digital-Design-2020-2021/blob/master/wks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ONLY FOR COLAB
# Not required in Binder

!wget -c https://repo.anaconda.com/miniconda/Miniconda3-4.5.4-Linux-x86_64.sh
!chmod +x Miniconda3-4.5.4-Linux-x86_64.sh
!bash ./Miniconda3-4.5.4-Linux-x86_64.sh -b -f -p /usr/local
!conda install -q -y --prefix /usr/local python=3.6 ujson

import sys
sys.path.append('/usr/local/lib/python3.6/site-packages')

import ujson
print(ujson.dumps({1:2}))

!conda install -c conda-forge igl
!conda install -c conda-forge meshplot

In [ ]:
!pip install --upgrade --force-reinstall Pillow

import igl
import scipy
import scipy as sp
import numpy as np
from meshplot import plot, subplot, interact
import meshplot
from scipy.sparse.linalg import eigs,eigsh
from scipy.sparse import csr_matrix
import os 
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

import tensorflow as tf
tf.__version__

In [ ]:
def plot_pair(v1, v2, f1, f2, c1, c2, color_ops = {}):
    # Compute a scale factor
    M1 = igl.massmatrix(v1, f1, igl.MASSMATRIX_TYPE_VORONOI)
    M2 = igl.massmatrix(v2, f2, igl.MASSMATRIX_TYPE_VORONOI)
    scale_factor = np.sqrt(np.sum(M2)/np.sum(M1))

    # Align the shapes
    v2 = v2 - np.mean(v2,axis=0)
    v1_align = v1 * scale_factor + np.mean(v1,axis=0) + [0.7,-0.7,0.0]

    # Merge the models
    v_all = np.vstack((v1_align, v2))
    f_all = np.vstack((f1, f2 + np.max(f1)+1))
    c_all = np.vstack((c1, c2))
    
    plot(v_all, f_all, c_all, shading = color_ops)

In [ ]:
# Load Shapes
v_src, f_src = igl.read_triangle_mesh(os.path.join('.', "data", "tr_reg_089.off"))

L_src = -igl.cotmatrix(v_src, f_src)
M_src = igl.massmatrix(v_src, f_src, igl.MASSMATRIX_TYPE_VORONOI)

try:
    evals_src, evecs_src = eigsh(L_src, 200, M_src, sigma=0.0, which='LM', maxiter=1e9, tol=1.e-15)
except:
    evals_src, evecs_src = eigsh(L_src- 1e-8* scipy.sparse.identity(v_src.shape[0]), 200,
                         M_src, sigma=0.0, which='LM', maxiter=1e9, tol=1.e-15)

evals_src = evals_src.astype(np.float32)
evals_src = evals_src.astype(np.float32)

v_tar, f_tar = igl.read_triangle_mesh(os.path.join('.', "data", "tr_reg_090.off"))

L_tar = -igl.cotmatrix(v_tar, f_tar)
M_tar = igl.massmatrix(v_tar, f_tar, igl.MASSMATRIX_TYPE_VORONOI)

try:
    evals_tar, evecs_tar = eigsh(L_tar, 200, M_tar, sigma=0.0, which='LM', maxiter=1e9, tol=1.e-15)
except:
    evals_tar, evecs_tar = eigsh(L_tar- 1e-8* scipy.sparse.identity(v_tar.shape[0]), 200,
                         M_src, sigma=0.0, which='LM', maxiter=1e9, tol=1.e-15)
    
evals_tar = evals_tar.astype(np.float32)
evals_tar = evals_tar.astype(np.float32)

# WKS

We will use the Wave Kernel Signature (WKS) descriptor to do the matching. Recall the formula:

$K_E(x,x) = \sum\limits_{l=1}^{\infty}e^{- \frac{(log(E) - log(\lambda_l))^2}{2\sigma^2}} \phi_l(x)^2 $

Where:
- $sigma = 7 \delta$
- $delta =  (e_{max} - e{min})/ M$
- $e_{max} = log(E_N) - 2\sigma$
- $e_{min} = log(E_1) + 2\sigma$
- $E_N$ is the max eigenvalue in absolute value
- $E_1$ is the min non-zero eigenvalue in absolute value
- $M$ is the number of WKS scales

The tasks are:
- Read the meshes, compute the LBO eigenvectors
- Define the WKS computation
- Visualize the WKS scales on meshes
- Perform the matching using WKS (Nearest-Neighbor in the descriptor space)
- Visualize the matching (and compute the error)

Are the descriptors coherent among the shapes, for different descriptor scales? Is the matching good? We can change the number of descriptors: does it impact the matching?

In [ ]:
def WKS(vertices, faces, evals, evecs, wks_size, variance):
    # Number of vertices
    n = vertices.shape[0]
    WKS = np.zeros((n,wks_size))

    # Just for numerical stability
    evals[evals<1e-6] = 1e-6

    # log(E)
    log_E = np.log(evals).T

    # Define the energies step
    e = np.linspace(log_E[1], np.max(log_E)/1.02, wks_size)

    # Compute the sigma
    sigma = (e[1]-e[0]) * variance
    C = np.zeros((wks_size,1))

    for i in np.arange(0,wks_size):
        # Computing WKS
        WKS[:,i] = np.sum(
            (evecs)**2 * np.tile( np.exp((-(e[i] - log_E)**2) / (2*sigma**2)),(n,1)), axis=1)
        
        # Noramlization
        C[i] = np.sum(np.exp((-(e[i]-log_E)**2)/(2*sigma**2)))
        
    WKS = np.divide(WKS,np.tile(C,(1,n)).T)
    return WKS

In [ ]:
# Computing the descriptors for the two shapes
d_src = WKS(v_src, f_src, evals_src, evecs_src, 100, 7)
d_tar = WKS(v_tar, f_tar, evals_tar, evecs_tar, 100, 7)

In [ ]:
# Visualizing descriptors
i = 2
plot_pair(v_src, v_tar, f_src, f_tar, d_src[:,i:i+1], d_tar[:,i:i+1])

In [ ]:
# Nearest Neighbor
treesearch = sp.spatial.cKDTree(d_tar)
p2p = treesearch.query(d_src, k=1)[1]

In [ ]:
# To see the quality of the matching we plot a function on one shape and we transfer it to the other
funz_ = (v_tar - np.min(v_tar,0))/np.tile((np.max(v_tar,0)-np.min(v_tar,0)),(np.size(v_tar,0),1));
colors = np.cos(funz_);
funz_tar = (colors-np.min(colors))/(np.max(colors) - np.min(colors));
funz_src = funz_tar[p2p]


In [ ]:
# Plot and (euclidean) error evaluation
funz_src = funz_tar[p2p]
plot_pair(v_src, v_tar, f_src, f_tar, funz_src,funz_tar)
err = np.sum(np.square(v_tar - v_tar[p2p,:]))
print(err)